In [10]:
import mlrose_hiive
import numpy as np
import itertools

rng = np.random.default_rng(seed=42)


In [11]:
weights = 1 + rng.integers(0, 50, size=30)
values = 1 + rng.integers(0, 50, size=30)

fitness_funcs = [
    ("OneMax", mlrose_hiive.OneMax()),
    ("FlipFlop", mlrose_hiive.FlipFlop()),
    ("Knapsack", mlrose_hiive.Knapsack(weights=weights, values=values))
]

In [12]:
for name, function in fitness_funcs:
    print(f"Finding best RHC hyper parameters for {name}")
    
    # Determine length of state vector based on problem
    if name == "OneMax":
        problem = mlrose_hiive.DiscreteOpt(length=200, fitness_fn=function, maximize=True, max_val=2)

    elif name == "FlipFlop":
        length = len(weights)
        problem = mlrose_hiive.FlipFlopGenerator().generate(seed=42, size=200)

    elif name == "Knapsack":
        length = len(values)
        problem = mlrose_hiive.KnapsackGenerator().generate(
            seed=42, 
            number_of_items_types=50, 
            max_item_count=30, 
            max_weight_per_item=100, 
            max_value_per_item=100, 
            max_weight_pct=0.8
        )

    best_restart_param = None
    best_restart_fitness_value = None
    
    for i in [0, 25, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]:        
        results = mlrose_hiive.random_hill_climb(problem, max_attempts = 500, max_iters=2048, curve=True, random_state=42, restarts = i)
        rhc_best_state, rhc_best_fitness, rhc_fitness_curve = results
        
        if not best_restart_fitness_value or rhc_best_fitness > best_restart_fitness_value:
            best_restart_param = i
            best_restart_fitness_value = rhc_best_fitness
            
    print(f"Best RHC hyper parameters for {name}:\n\t restarts: {best_restart_param}\n")

Finding best RHC hyper parameters for TSP
Best RHC hyper parameters for TSP:
	 restarts: 0

Finding best RHC hyper parameters for FlipFlop
Best RHC hyper parameters for FlipFlop:
	 restarts: 300

Finding best RHC hyper parameters for Knapsack
Best RHC hyper parameters for Knapsack:
	 restarts: 75


In [18]:
# SA Tuning

# init_temp, decay, min_temp
sa_hyperparams = [
   [1, 2, 4, 8, 16, 32, 64, 128, 256],
   [0.1, 0.2, 0.4, 0.8, 1.0],
   [0.001, 0.01, 0.1, 1]
]


for name, function in fitness_funcs:
    print(f"Finding best SA hyper parameters for {name}")
    
    # Determine length of state vector based on problem
    if name == "OneMax":
        problem = mlrose_hiive.DiscreteOpt(length=200, fitness_fn=function, maximize=True, max_val=2)

    elif name == "FlipFlop":
        length = len(weights)
        problem = mlrose_hiive.FlipFlopGenerator().generate(seed=42, size=200)

    elif name == "Knapsack":
        length = len(values)
        problem = mlrose_hiive.KnapsackGenerator().generate(
            seed=42, 
            number_of_items_types=50, 
            max_item_count=30, 
            max_weight_per_item=100, 
            max_value_per_item=100, 
            max_weight_pct=0.8
        )
    
    best_sa_param = None
    best_sa_fitness_value = None
    
    for i in itertools.product(*sa_hyperparams):
        
        decay = mlrose_hiive.ExpDecay(init_temp = i[0], exp_const=i[1], min_temp=i[2])
        
        results = mlrose_hiive.simulated_annealing(problem, max_attempts=500, max_iters=2048, curve=True, random_state=42, schedule=decay)
        sa_best_state, sa_best_fitness, sa_fitness_curve = results
        
        if not best_sa_fitness_value or sa_best_fitness > best_sa_fitness_value:
            best_sa_param = i
            best_sa_fitness_value = sa_best_fitness
            
    init_temp, exp_const, min_temp = best_sa_param
    print(f"Best SA hyper parameters for {name}:\n\tinit_temp: {init_temp}\n\texp_const: {exp_const}\n\tmin_temp: {min_temp}\n")

Finding best SA hyper parameters for TSP
Best SA hyper parameters for TSP:
	init_temp: 256
	exp_const: 0.1
	min_temp: 1

Finding best SA hyper parameters for FlipFlop
Best SA hyper parameters for FlipFlop:
	init_temp: 256
	exp_const: 0.1
	min_temp: 0.001

Finding best SA hyper parameters for Knapsack
Best SA hyper parameters for Knapsack:
	init_temp: 128
	exp_const: 0.1
	min_temp: 0.001


In [14]:
# GA Tuning

# pop_size, mutation_prob
ga_hyperparams = [
   [100, 200, 400, 800, 1000],
   [0.2, 0.4, 0.5, 0.8, 1.0]
]


for name, function in fitness_funcs:
    best_ga_param = None
    best_ga_fitness_value = None
    
    print(f"Finding best GA hyper parameters for {name}")
    
    # Determine length of state vector based on problem
    if name == "OneMax":
        problem = mlrose_hiive.DiscreteOpt(length=200, fitness_fn=function, maximize=True, max_val=2)

    elif name == "FlipFlop":
        length = len(weights)
        problem = mlrose_hiive.FlipFlopGenerator().generate(seed=42, size=200)

    elif name == "Knapsack":
        length = len(values)
        problem = mlrose_hiive.KnapsackGenerator().generate(
            seed=42, 
            number_of_items_types=50, 
            max_item_count=30, 
            max_weight_per_item=100, 
            max_value_per_item=100, 
            max_weight_pct=0.8
        )

    for i in itertools.product(*ga_hyperparams):
        results = mlrose_hiive.genetic_alg(problem, max_attempts=500, max_iters=2048, curve=True, random_state=42,pop_size=i[0],mutation_prob=i[1])
        ga_best_state, ga_best_fitness, ga_fitness_curve = results
        
        if not best_ga_fitness_value or ga_best_fitness > best_ga_fitness_value:
            best_ga_param = i
            best_ga_fitness_value = ga_best_fitness
    pop_size, mutation_prob = best_ga_param
    print(f"Best GA hyper parameters for {name}:\n\tpop_size: {pop_size}\n\tmutation_prob: {mutation_prob}\n")

Finding best GA hyper parameters for TSP
Best GA hyper parameters for TSP:
	pop_size: 100
	mutation_prob: 0.8

Finding best GA hyper parameters for FlipFlop
Best GA hyper parameters for FlipFlop:
	pop_size: 800
	mutation_prob: 0.8

Finding best GA hyper parameters for Knapsack
Best GA hyper parameters for Knapsack:
	pop_size: 400
	mutation_prob: 1.0


In [15]:
# MIMIC Tuning

mimic_hyperparams = [
   [100, 200, 400, 800, 1000],
   [0.2, 0.4, 0.5, 0.8, 1.0]
]
for name, function in fitness_funcs:
    best_mimic_param = None
    best_mimic_fitness_value = None
    
    print(f"Finding best MIMIC hyper parameters for {name}")
    
    # Determine length of state vector based on problem
    if name == "OneMax":
        problem = mlrose_hiive.DiscreteOpt(length=200, fitness_fn=function, maximize=True, max_val=2)

    elif name == "FlipFlop":
        length = len(weights)
        problem = mlrose_hiive.FlipFlopGenerator().generate(seed=42, size=200)

    elif name == "Knapsack":
        length = len(values)
        problem = mlrose_hiive.KnapsackGenerator().generate(
            seed=42, 
            number_of_items_types=50, 
            max_item_count=30, 
            max_weight_per_item=100, 
            max_value_per_item=100, 
            max_weight_pct=0.8
        )

    for i in itertools.product(*mimic_hyperparams):
        results = mlrose_hiive.mimic(problem, max_attempts = 100, max_iters = 2048,  curve = True, random_state = 42, pop_size=i[0], keep_pct=i[1])
        mimic_best_state, mimic_best_fitness, mimic_fitness_curve = results
        
        if not best_mimic_fitness_value or mimic_best_fitness > best_mimic_fitness_value:
            best_mimic_param = i
            best_mimic_fitness_value = mimic_best_fitness
    mimic_pop_size, keep_pct = best_mimic_param
    print(f"Best MIMIC hyper parameters for {name}:\n\tpop_size: {mimic_pop_size}\n\keep_pct: {keep_pct}\n")

Finding best MIMIC hyper parameters for TSP
Best MIMIC hyper parameters for TSP:
	pop_size: 100
\keep_pct: 0.2

Finding best MIMIC hyper parameters for FlipFlop
Best MIMIC hyper parameters for FlipFlop:
	pop_size: 800
\keep_pct: 0.2

Finding best MIMIC hyper parameters for Knapsack
Best MIMIC hyper parameters for Knapsack:
	pop_size: 1000
\keep_pct: 0.5
